# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,-4.42,87,100,0.82,CL,1690230691
1,1,tiksi,71.6872,128.8694,3.75,82,20,1.98,RU,1690230691
2,2,waitangi,-43.9535,-176.5597,12.12,98,100,2.24,NZ,1690230692
3,3,vanimo,-2.6741,141.3028,24.97,89,51,1.33,PG,1690230693
4,4,susuman,62.7833,148.1667,14.84,57,28,1.43,RU,1690230693


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_city_df = city_data_df[(city_data_df['Cloudiness'] <=10.0) & (city_data_df['Wind Speed'] <= 10.0) &\
                        (city_data_df['Humidity'] >= 60) & (city_data_df['Humidity'] < 100) \
                                     & (city_data_df['Max Temp'] <= 30) & (city_data_df['Max Temp'] >22)]

# Drop any rows with null values
city_data_clean_df = ideal_weather_city_df.dropna()

# Display sample data
city_data_clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,pisco,-13.7000,-76.2167,23.03,60,0,7.20,PE,1690230698
146,146,pacasmayo,-7.4006,-79.5714,23.29,72,1,7.57,PE,1690230750
156,156,afaahiti,-17.7500,-149.2833,27.93,81,6,9.11,PF,1690230754
174,174,constantia,44.1833,28.6500,23.38,73,0,7.98,RO,1690230765
185,185,ryotsu-minato,38.0833,138.4333,25.88,78,1,1.80,JP,1690230769
190,190,carutapera,-1.1950,-46.0200,25.00,91,4,3.03,BR,1690230771
203,203,fale old settlement,-9.3852,-171.2468,28.80,72,7,7.56,TK,1690230775
217,217,arraial do cabo,-22.9661,-42.0278,22.98,78,0,8.75,BR,1690230781
237,237,agat village,13.3886,144.6585,28.52,90,0,4.63,GU,1690230791
305,305,belmonte,-15.8631,-38.8828,22.39,78,0,4.75,BR,1690230819


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_clean_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
18,pisco,PE,-13.7000,-76.2167,60,
146,pacasmayo,PE,-7.4006,-79.5714,72,
156,afaahiti,PF,-17.7500,-149.2833,81,
174,constantia,RO,44.1833,28.6500,73,
185,ryotsu-minato,JP,38.0833,138.4333,78,
190,carutapera,BR,-1.1950,-46.0200,91,
203,fale old settlement,TK,-9.3852,-171.2468,72,
217,arraial do cabo,BR,-22.9661,-42.0278,78,
237,agat village,GU,13.3886,144.6585,90,
305,belmonte,BR,-15.8631,-38.8828,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
categories = 'accommodation.hotel'
radius = 10000

params = {
    "categories":categories,
    "limit":1,
    "lang":'en',
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pisco - nearest hotel: La Portada
pacasmayo - nearest hotel: Playa Linda
afaahiti - nearest hotel: Omati Lodge
constantia - nearest hotel: Hotel Chérica
ryotsu-minato - nearest hotel: ゆたかや旅館
carutapera - nearest hotel: No hotel found
fale old settlement - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
agat village - nearest hotel: Navy Getway Inns & Suites
belmonte - nearest hotel: No hotel found
cabedelo - nearest hotel: No hotel found
trairi - nearest hotel: Pousada Villa Aurora
ad dilinjat - nearest hotel: No hotel found
kirakira - nearest hotel: No hotel found
ribeira do pombal - nearest hotel: No hotel found
sao jose da coroa grande - nearest hotel: Hotel do Sol
novofedorivka - nearest hotel: Оазис
tongchuanshi - nearest hotel: No hotel found
shimoda - nearest hotel: ホテルウラガ
fortaleza - nearest hotel: Hotel Catedral
rosetta - nearest hotel: No hotel found
vegreville - nearest hotel: Hotello
sao joao da barra - nearest hotel: Pous

,City,Country,Lat,Lng,Humidity,Hotel Name
18,pisco,PE,-13.7000,-76.2167,60,La Portada
146,pacasmayo,PE,-7.4006,-79.5714,72,Playa Linda
156,afaahiti,PF,-17.7500,-149.2833,81,Omati Lodge
174,constantia,RO,44.1833,28.6500,73,Hotel Chérica
185,ryotsu-minato,JP,38.0833,138.4333,78,ゆたかや旅館
190,carutapera,BR,-1.1950,-46.0200,91,No hotel found
203,fale old settlement,TK,-9.3852,-171.2468,72,No hotel found
217,arraial do cabo,BR,-22.9661,-42.0278,78,No hotel found
237,agat village,GU,13.3886,144.6585,90,Navy Getway Inns & Suites
305,belmonte,BR,-15.8631,-38.8828,78,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# %%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 750,
    frame_height = 600,
    size = 60.0,
    scale = 0.5,
    color = "City",
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)